# Hypothesis Testing Demonstration

In [ ]:
import pandas as pd
import numpy as np

Our dataset comes from Kaggle and contains information about the student, including their identity, education level, and test scores. Let's load it in using pandas and display the first couple of rows. 

In [ ]:
scores = pd.read_csv("StudentsPerformance (3).csv")
scores.head(5)


In [ ]:
len(scores)

The 'test preparation course' column contains the unique values 'none' and 'completed', depending on whether or not the student has completed test preparation. 

In [ ]:
test_prep = scores['test preparation course']
test_prep.head(5)

The 'writing score' column contains individual data on student writing  scores

In [ ]:
math_scores = scores["math score"]
math_scores.head(5)

Let's split up the test preparation course column into two series, one with those students who completed test prep and those who didn't

In [ ]:
completed_scores = scores[scores['test preparation course'] == 'completed']['math score']
not_completed_scores = scores[scores['test preparation course'] == 'none']['math score']

Let's visualize the math scores between those who completed test prep versus those who didn't. Here are the two histograms:

In [ ]:
import seaborn as sns
sns.distplot(completed_scores, color="skyblue", label="Test Prep Completed", bins = 30)
sns.distplot(not_completed_scores, color="green", label="Test Prep Not Completed", bins = 30)

## Null and Alternative Hypotheses

**We would like to see whether whether test preparation was associated with math scores. To set up our hypothesis test, we can represent the two views of the world using the following null and alternative hypotheses:**

Null hypothesis: In the population, the distribution of math scores for those who completed test prep is the same for those who didn't complete test prep. The difference in the sample is due to chance.

Alternative hypothesis: In the population, the students who completed test preparation have higher math scores, on average, than those who didn't complete test prep. The observed difference is "real". This is what would be considered a one-sided alternative hypothesis. (Two-sided would be something like "the distributions are different, not necessarily greater or less than")

## Test Statistic

We want our test statistic to be the difference between the mean completed score and mean none score based on our original data. The value below is our "observed value" of the test statistic. Large positive values of this statistic will favor the alternative hypothesis. 

In [ ]:
observed_difference = np.mean(completed_scores) - np.mean(not_completed_scores)
observed_difference 

## Permutation Testing

If there were really no difference between the two distributions in the underlying population, then whether each test preparation was completed or not by the student shouldn't affect their math score, therefore should make no difference to the average. 

To investigate this more, we will simulate the test statistic under the null hypothesis. Let's shuffle all the math scores randomly among the students.

We conduct this random permutation below.

In [ ]:
def shuffle(series):
    '''
    Shuffles a series and resets index to preserve shuffle when adding series
    back to DataFrame.
    '''
    return series.sample(frac=1, replace=False).reset_index(drop=True)

In [ ]:
scores["Shuffled"] = shuffle(scores["math score"])
scores.head()

In the cell below, we will simulate our test statistic many times and collect the differences in an array.

In [ ]:
#Permutation Test
differences = np.array([])

repetitions = 5000
for i in np.arange(repetitions):
    scores["Shuffled"] = shuffle(scores["math score"])
  
    # Find the difference between the means of two randomly assigned groups
    none = scores[scores['test preparation course'] == 'none']['Shuffled']
    completed = scores[scores['test preparation course'] == 'completed']['Shuffled']
    simulated_difference = np.mean(completed) - np.mean(none)

    differences = np.append(differences, simulated_difference)

Here's a histogram of the simulated difference in means below. We see that majority of the values seem to favor 0:

In [ ]:
sns.distplot(differences)

## P-Value Calculation + Conclusion

Let us now calculate the p-value in order to make a conclusion for this hypothesis test. The p-value in this case would be the proportion of values under the sampling distribution of simulated differences above that are larger or equal to the observed difference. 

In [ ]:
p_value = np.count_nonzero(differences >= observed_difference) / repetitions
p_value

Due to our p-value being less than the threshold of 0.05, we reject the null hypothesis. 

And that concludes our introductory workshop on hypothesis testing! Hope you find this useful in your projects and in the near future! Happy coding :)